In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import sys

from pymoo.indicators.igd import IGD
from pymoo.indicators.igd_plus import IGDPlus
from pymoo.indicators.hv import HV

In [2]:
res_col_subset=["staticCollision", "dynamicCollision", "fuzzingOperationTimes"]

In [3]:
# https://stackoverflow.com/questions/68522283/removing-dominated-rows-from-a-pandas-dataframe-rows-with-all-values-lower-th
def get_dominants(df):
    seen = [] # keep track of tested rows
    idx = df.index # initial index
    for i in range(len(df)+1):
        x = idx[0]
        if x in seen:   # done a full loop
            return df.loc[idx]
        seen.append(idx[0])
        # check which rows are dominated and drop them from the index
        idx = (df.loc[idx]-df.loc[x]).le(0).all(axis=1)
        # put tested row at the end
        idx = list(idx[~idx].index)+[x]

def dominant_subset_cols(df):
    return get_dominants(df[res_col_subset])

In [4]:
def invert_signs_for_metrics(df):
    df_new = df.copy()
    df_new.staticCollision *= -1
    df_new.dynamicCollision *= -1
    return df_new

In [5]:
condbased_res960 = pd.read_csv("steppingcell-condbased-tts-cutdown", delim_whitespace=True)
condbased_res240 = pd.read_csv("tts-history-condfuzz-240", delim_whitespace=True)
condbased_res480 = pd.read_csv("tts-history-condfuzz-480", delim_whitespace=True)
condbased_res720 = pd.read_csv("tts-history-condfuzz-720", delim_whitespace=True)
dcondbased_240 = invert_signs_for_metrics(dominant_subset_cols(condbased_res240))
dcondbased_480 = invert_signs_for_metrics(dominant_subset_cols(condbased_res480))
dcondbased_720 = invert_signs_for_metrics(dominant_subset_cols(condbased_res720))
dcondbased_960 = invert_signs_for_metrics(dominant_subset_cols(condbased_res960))

In [6]:
dcondbased_960

,staticCollision,dynamicCollision,fuzzingOperationTimes
0,-180.0,-0.0,294.32
3,-6.0,-52.0,161.70
4,-17.0,-0.0,316.80
7,-67.0,-2.0,108.00
9,-10.0,-44.0,163.30
11,-2.0,-33.0,283.62
12,-57.0,-40.0,129.38
16,-24.0,-17.0,253.30
19,-48.0,-13.0,253.50
23,-38.0,-8.0,297.18


In [7]:
def max_point_from_front(pf):
    return np.array([np.max(pf.staticCollision), np.max(pf.dynamicCollision), np.max(pf.fuzzingOperationTimes)])

In [8]:
def max_point_from_all_fronts(pf_list):
    max_points_2d = np.column_stack(list(map(lambda pf: max_point_from_front(pf), pf_list)))
    max_of_all = np.amax(max_points_2d, axis=1)
    return max_of_all

In [9]:
def front_to_numpy_array(front):
    return front[res_col_subset].to_numpy()

In [10]:
all_fronts = [dcondbased_240, dcondbased_480, dcondbased_720, dcondbased_960]
max_point_from_all_fronts(all_fronts)

array([ -0. ,  -0. , 316.8])

In [11]:
def indicators_for_intermediate_fronts(fronts):
    # Reference point is obtained as the max point from all fronts
    ref_point = max_point_from_all_fronts(fronts)
    ref_front = front_to_numpy_array(dcondbased_960)

    for front_pandas in fronts:
        # Get front as numpy array
        front = front_to_numpy_array(front_pandas)
        hv_ind = HV(ref_point=ref_point)
        igd_ind = IGD(ref_front)
        igdplus_ind = IGDPlus(ref_front)
        hv_val = hv_ind(front)
        igd_val = igd_ind(front)
        igdplus_val = igdplus_ind(front)
        print("HV = " + str(hv_val) + ", IGD = " + str(igd_val) + ", IGDPlus =  " + str(igdplus_val))

In [12]:
indicators_for_intermediate_fronts(all_fronts)

HV = 15218.940000000002, IGD = 97.3926962616355, IGDPlus =  31.31434144820441
HV = 158384.26, IGD = 42.360298021372074, IGDPlus =  10.473863455701686
HV = 433930.92000000004, IGD = 7.07744453846945, IGDPlus =  1.6
HV = 447554.12000000005, IGD = 0.0, IGDPlus =  0.0
